<a href="https://colab.research.google.com/github/saksham1211/Marketingai_Internship/blob/master/Topic_Modeling_for_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 16.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=64a546f8377fc3c848dbed9975965584aa081aa77722beb50e7882fa752b9bfa
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=a965198054cf0043672dc32bc86250cd9c3de9cd09f36f929a6eb5d1c31e5bf0
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import random
import re, nltk, spacy, gensim
import pyLDAvis
import pyLDAvis.sklearn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 50)

In [ ]:
df = pd.read_csv('/content/Book2.csv', encoding="latin-1")
df.set_index('CATEGORY', inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['DESCRIPTION'].values.astype('U'))
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df.index)
def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_category = []
    
   
    idx = indices[indices == name].index[0]

    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    top_10_indexes = list(score_series.iloc[1:6].index)

    for i in top_10_indexes:
        recommended_category.append(list(df.index)[i])
        
    return recommended_category

In [ ]:
recommendations('Schwab_IRA')

['Schwab_IRA', 'Schwab_IRA', 'Schwab_IRA', 'Schwab_IRA', 'SBI']

In [ ]:
recommendations('ICICI_LOMBARD')

['ICICI_LOMBARD',
 'ICICI_LOMBARD',
 'ICICI_LOMBARD',
 'ICICI_LOMBARD',
 'ICICI_LOMBARD']

In [ ]:
df.loc['Schwab_IRA'].DESCRIPTION

CATEGORY
Schwab_IRA    Distributions from your IRA are generally subj...
Schwab_IRA    Your tax withholding election will remain in e...
Schwab_IRA    Withholding income taxes may subject you to IR...
Schwab_IRA    Please select a withholding instruction for th...
Schwab_IRA    State tax withholding may be required from you...
Schwab_IRA    While Schwab makes every effort to obtain info...
Schwab_IRA    A Schwab Qualified Plan is a Schwab Profit Sha...
Schwab_IRA    The Cash Features Program is a service that of...
Schwab_IRA    You understand that additional information abo...
Schwab_IRA    The Bank Sweep feature is designated as the Ca...
Schwab_IRA    Your deposits at each Sweep Bank are insured b...
Schwab_IRA    You understand and agree that Schwab may (1) m...
Schwab_IRA    Schwab will notify you in writing of changes t...
Schwab_IRA    You will receive account statements, trade con...
Schwab_IRA    For complete information, please see Important...
Schwab_IRA    If you already ha

In [ ]:
df.loc["ICICI_LOMBARD"].DESCRIPTION

CATEGORY
ICICI_LOMBARD    DuringÂ FY2019, we settledÂ 99.87 %Â health in...
ICICI_LOMBARD    Well, we donât wish to blow our own trumpet,...
ICICI_LOMBARD    ICRA (an Associate of Moody's Investors Servic...
ICICI_LOMBARD    The PMFBY was launched in 2016 and replaces al...
ICICI_LOMBARD    Commercial term that buyer shall accept delive...
ICICI_LOMBARD    Any one accident. The term, in relation to lia...
ICICI_LOMBARD    Any one vessel. The term, which relates to Mar...
ICICI_LOMBARD    An expert in statistics and a mathematician in...
ICICI_LOMBARD    A branch of knowledge which deals with mathema...
ICICI_LOMBARD    Any event not caused or contributed to by man....
ICICI_LOMBARD    A clause included in Excess of Loss Treaties p...
ICICI_LOMBARD    This refers to the expenses incurred by the Di...
ICICI_LOMBARD    Acquisition cost incurred by the direct insure...
ICICI_LOMBARD    Condition characterized by illness indicative ...
ICICI_LOMBARD    As there are different methods of ac

In [ ]:
df.loc["Merrilllynch_CRA"].DESCRIPTION

CATEGORY
Merrilllynch_CRA    Use this form to transfer all or a portion of ...
Merrilllynch_CRA    For best results, please complete the informat...
Merrilllynch_CRA    Many firms transfer assets using an automated ...
Merrilllynch_CRA    Remember to update any automatic deposits or w...
Merrilllynch_CRA    Some transfers may result in tax consequences,...
Merrilllynch_CRA    If youâre rolling over assets from a 401(k) ...
Merrilllynch_CRA    Before indicating which assets to transfer, yo...
Merrilllynch_CRA    If you are transferring mutual funds directly ...
Merrilllynch_CRA    Most financial institutions require either wri...
Merrilllynch_CRA    Use the sections below to list the assets you ...
Merrilllynch_CRA    Use the sections below to list the assets you ...
Merrilllynch_CRA    Â£Â£ Immediately liquidate CDs and transfer in...
Merrilllynch_CRA    Securities include stocks, bonds, exchange tra...
Merrilllynch_CRA    Transfer your account to Merrill Lynch as you ...
Merrilllync

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df = pd.concat([pd.Series(str(row['CATEGORY']), str(row['DESCRIPTION']).split('. '))              
                    for _, row in df.iterrows()]).reset_index()
df.columns = ['sentence', 'CATEGORY']
df['sentence'] = df['sentence'].map(lambda x: re.sub(r'\W+', ' ', x))
print('We have ', len(df), 'sentences in total')

We have  358 sentences in total


In [ ]:
df.loc[df['CATEGORY'] == 'Merrilllynch_CRA']

,sentence,CATEGORY
0,Use this form to transfer all or a portion of ...,Merrilllynch_CRA
1,For best results please complete the informati...,Merrilllynch_CRA
2,If you need more room for information or signa...,Merrilllynch_CRA
3,Use one form for each account that youâ re tra...,Merrilllynch_CRA
4,Many firms transfer assets using an automated ...,Merrilllynch_CRA
...,...,...
69,MLPF S is a registered broker dealer Member SI...,Merrilllynch_CRA
70,Next with the expertise of our Chief Investmen...,Merrilllynch_CRA
71,Wealth management is an ongoing process,Merrilllynch_CRA
72,You ll meet with your advisor often and have i...,Merrilllynch_CRA


In [ ]:
df.loc[df['CATEGORY'] == 'ICICI_LOMBARD']

,sentence,CATEGORY
206,DuringÂ FY2019 we settledÂ 99 87 Â health insu...,ICICI_LOMBARD
207,Well we donâ t wish to blow our own trumpet bu...,ICICI_LOMBARD
208,ICRA an Associate of Moody s Investors Service...,ICICI_LOMBARD
209,The PMFBY was launched in 2016 and replaces al...,ICICI_LOMBARD
210,The scheme has extended coverage under localiz...,ICICI_LOMBARD
211,and aims at adoption of technology for the pur...,ICICI_LOMBARD
212,Through increased farmer awareness and low far...,ICICI_LOMBARD
213,Commercial term that buyer shall accept delive...,ICICI_LOMBARD
214,In General Insurance this term is used mostly ...,ICICI_LOMBARD
215,Any one accident,ICICI_LOMBARD


In [ ]:
a = 0
for i in range(a,a+8):
    print(df.sentence[i])
    print()

Use this form to transfer all or a portion of your account from another financial institution to your Merrill Lynch Pierce Fenner Smith Incorporated â Merrill Lynchâ account

For best results please complete the information requested below in parts 1 to 4 using ALL CAPITAL letters and black ink

If you need more room for information or signatures use a copy of the relevant page or a blank sheet

Use one form for each account that youâ re transferring

Many firms transfer assets using an automated transfer process which typically takes 5 to 6 business days

Other firms either donâ t use the automated process or must process some assets manually which takes more time

The following estimated time frames for manual transfers are from the date of receipt of instructions by the delivering firm and may vary greatly

Remember to update any automatic deposits or withdrawals youâ ve set up on the account youâ re transferring to Merrill Lynch



In [ ]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=3000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(df['sentence'])

In [ ]:
lda_model = LatentDirichletAllocation(n_components=40, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=40, n_jobs=-1,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster      Freq
topic                                                 
28     23.606779 -27.819445       1        1  9.149717
2      -1.468278  24.632059       2        1  8.462422
24    -23.649248 -23.484100       3        1  8.459028
36     13.156273  22.572628       4        1  7.697677
3     -14.081748 -31.640413       5        1  6.444641
38     -1.227611 -35.868015       6        1  4.926941
32     -5.402138  14.418217       7        1  4.299334
31     14.742131 -21.958942       8        1  4.045790
22    -16.458038  18.191874       9        1  3.856107
20     11.497930 -33.042763      10        1  3.835283
35    -28.870626  -8.157530      11        1  3.557213
0       5.548925  14.480577      12        1  3.259356
4     -13.078332 -20.434235      13        1  3.013782
37    -25.279318   6.024581      14        1  2.839777
12     24.661472 -17.884966      15        1  2.753724
39     22.031687  -7.511183      16        1  2.480830
7      31.194286  -1.937071      17        1  2.144044
6     -13.715621   7.719382      18        1  1.953994
23      5.712842 -24.095497      19        1  1.787581
13     21.865992   3.375811      20        1  1.678724
11     18.169292  12.025668      21        1  1.608162
25     -4.852035 -25.183731      22        1  1.350700
16    -19.274153 -13.019238      23        1  1.333010
33     10.721196   6.474187      24        1  1.099369
15     14.317254 -12.956407      25        1  1.059987
30    -18.574400  -1.731525      26        1  0.954209
8     -12.294326  -8.340729      27        1  0.813456
14     -9.703804  -0.866392      28        1  0.723220
29     14.365800  -1.627624      29        1  0.692389
27     -0.442288 -18.155491      30        1  0.445794
21      2.384781   6.050657      31        1  0.428188
1      -7.166992 -13.938316      32        1  0.425158
9      -4.872026   4.710040      33        1  0.348119
19      6.399309  -0.309467      34        1  0.313726
5      -0.781342  -0.965754      35        1  0.293091
26     -0.588244 -11.170997      36        1  0.293091
17      2.877829  -6.176871      37        1  0.293091
34      5.979822 -13.714424      38        1  0.293091
10      9.190524  -7.186387      39        1  0.293091
18     -4.767587  -6.154807      40        1  0.293091, topic_info=            Term       Freq      Total Category  logprob  loglift
333     transfer  41.000000  41.000000  Default  30.0000  30.0000
39          bank  44.000000  44.000000  Default  29.0000  29.0000
7        account  79.000000  79.000000  Default  28.0000  28.0000
320          tax  30.000000  30.000000  Default  27.0000  27.0000
166  information  14.000000  14.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
297         send   0.022259   7.116958  Topic40  -5.7653   0.0650
316      subject   0.021823   5.291141  Topic40  -5.7851   0.3416
148         help   0.021939   8.955089  Topic40  -5.7798  -0.1793
225  outstanding   0.021600   6.852242  Topic40  -5.7954   0.0728
280       retail   0.021603   7.191764  Topic40  -5.7952   0.0246

[1825 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         5  0.248690          000
0        12  0.248690          000
0        30  0.248690          000
1         2  0.658819          15g
2         1  0.197802          401
...     ...       ...          ...
351      18  0.066440  withholding
352       6  0.614757        world
353       2  0.267023      writing
353       4  0.267023      writing
353      15  0.267023      writing

[1103 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[29, 3, 25, 37, 4, 39, 33, 32, 23, 21, 36, 1, 5, 38, 13, 40, 8, 7, 24, 14, 12, 26, 17, 34, 16, 31, 9, 15, 30, 28, 22, 2, 10, 20, 6, 27, 18, 35, 11, 19])

In [ ]:
# Show top 20 keywords for each topic
import numpy as np

def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
Topic 0,account,distribution,fund,funds,does,balance,mutual,available,schwab,shall,transferring,401,accounts,indians,000,cover,withholding,directly,require,note
Topic 1,held,form,transfer,firm,cash,authorization,individuals,distribution,ensure,sign,unique,class,details,instruct,number,selecting,injury,core,informed,days
Topic 2,tax,withholding,state,income,payment,financial,schwab,regarding,rules,subject,election,15g,form,submitted,federal,penalties,consult,consequences,result,applicable
Topic 3,deposit,bank,term,deposits,account,certificate,penalty,online,withdrawal,rate,lacs,tenors,premature,sweep,transferring,date,holds,senior,retail,customer
Topic 4,account,form,use,lynch,merrill,transfer,custodian,banking,goals,technology,needs,process,successor,accounts,sign,digital,core,set,related,documents
Topic 5,attached,access,revoke,india,received,accounts,include,set,citizens,makes,directly,forms,basis,program,custodian,held,right,fund,disclosure,penalty
Topic 6,rate,health,plan,sbi,security,withholding,medical,insurance,times,pin,details,card,using,payable,online,instantly,transactions,applicable,sponsored,401
Topic 7,customer,banking,business,required,distribution,retail,based,core,product,months,bank,corporate,businesses,package,build,services,thomas,healthy,cook,terms
Topic 8,trading,skip,authorization,checks,disbursement,sent,communications,type,individuals,new,capital,losing,ameritrade,health,shall,various,federal,refers,credit,regarding
Topic 9,citizens,safety,complete,section,assets,depending,minimum,tenors,different,trades,partial,place,application,package,scheme,accepted,select,portfolio,requested,consult


In [ ]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(40)]

# index names
docnames = ["Doc" + str(i) for i in range(0,358)]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [ ]:
df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39,dominant_topic
Doc0,0.00,0.00,0.00,0.00,0.92,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4
Doc1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.68,0.00,0.00,0.00,0.00,0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11
Doc2,0.01,0.01,0.01,0.01,0.20,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.21,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.20,0.01,0.20,0.01,0.01,0.01,0.01,16
Doc3,0.01,0.01,0.01,0.01,0.52,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.29,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,4
Doc4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,31
Doc5,0.00,0.00,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4
Doc6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,25
Doc7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,28
Doc8,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2
Doc9,0.00,0.00,0.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2


In [ ]:
df_document_topic.reset_index(inplace=True)
df_sent_topic= pd.merge(df, df_document_topic, left_index=True, right_index=True)
df_sent_topic.drop('index', axis=1, inplace=True)

In [ ]:
df_sent_topic.head(10)

,sentence,CATEGORY,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39,dominant_topic
0,Use this form to transfer all or a portion of ...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.92,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4
1,For best results please complete the informati...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.68,0.00,0.00,0.00,0.00,0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11
2,If you need more room for information or signa...,Merrilllynch_CRA,0.01,0.01,0.01,0.01,0.20,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.21,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.20,0.01,0.20,0.01,0.01,0.01,0.01,16
3,Use one form for each account that youâ re tra...,Merrilllynch_CRA,0.01,0.01,0.01,0.01,0.52,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.29,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,4
4,Many firms transfer assets using an automated ...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,31
5,Other firms either donâ t use the automated pr...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4
6,The following estimated time frames for manual...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,25
7,Remember to update any automatic deposits or w...,Merrilllynch_CRA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,28
8,Some transfers may result in tax consequences ...,Merrilllynch_CRA,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2
9,Consult your tax and financial professionals b...,Merrilllynch_CRA,0.00,0.00,0.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2


In [ ]:
df_sent_topic.tail(10)

,sentence,CATEGORY,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39,dominant_topic
348,Payment of interest at Monthly Quarterly Calen...,SBI,0.01,0.01,0.76,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,2
349,Payment of Monthly interest will be at discoun...,SBI,0.01,0.01,0.76,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,2
350,Interest will be paid at the contracted rate i...,SBI,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.80,0.01,0.01,0.01,0.01,0.01,0.01,0.01,32
351,Senior Citizens get 0 25 extra interest rate f...,SBI,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.80,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,27
352,For Retail Term Deposit up to Rs 5 00 lacs the...,SBI,0.00,0.00,0.00,0.89,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
353,For Retail Term Deposits above Rs 5 00 lacs bu...,SBI,0.00,0.00,0.00,0.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
354,Automatic renewals will take place only where ...,SBI,0.00,0.00,0.00,0.27,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.65,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,25
355,Park your lump sum amount in an SBI Term Depos...,SBI,0.00,0.00,0.00,0.48,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.43,0.00,0.00,0.00,3
356,TDS at prevalent Income tax rate is deducted i...,SBI,0.00,0.00,0.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2
357,Instructions for disposal of maturity proceeds...,SBI,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.89,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22


In [ ]:
df_sent_topic.loc[df_sent_topic['CATEGORY'] == 'Merrilllynch_CRA'][['sentence', 'dominant_topic']]

,sentence,dominant_topic
0,Use this form to transfer all or a portion of ...,4
1,For best results please complete the informati...,11
2,If you need more room for information or signa...,16
3,Use one form for each account that youâ re tra...,4
4,Many firms transfer assets using an automated ...,31
...,...,...
69,MLPF S is a registered broker dealer Member SI...,30
70,Next with the expertise of our Chief Investmen...,39
71,Wealth management is an ongoing process,31
72,You ll meet with your advisor often and have i...,15


In [ ]:
df_sent_topic.loc[df_sent_topic['CATEGORY'] == 'ICICI_LOMBARD'][['sentence', 'dominant_topic']]

,sentence,dominant_topic
206,DuringÂ FY2019 we settledÂ 99 87 Â health insu...,32
207,Well we donâ t wish to blow our own trumpet bu...,32
208,ICRA an Associate of Moody s Investors Service...,13
209,The PMFBY was launched in 2016 and replaces al...,32
210,The scheme has extended coverage under localiz...,31
211,and aims at adoption of technology for the pur...,4
212,Through increased farmer awareness and low far...,32
213,Commercial term that buyer shall accept delive...,25
214,In General Insurance this term is used mostly ...,22
215,Any one accident,0


In [ ]:
df_sent_topic.loc[df_sent_topic['dominant_topic'] == 4][['sentence', 'dominant_topic']].sample(4)

,sentence,dominant_topic
211,and aims at adoption of technology for the pur...,4
5,Other firms either donâ t use the automated pr...,4
63,All account holders or authorized signers of a...,4
120,Do not use this form to transfer between TD Am...,4


In [ ]:
df_sent_topic.loc[df_sent_topic['dominant_topic'] == 12][['sentence', 'dominant_topic']].sample(4)

,sentence,dominant_topic
272,Here are details on how to use ourÂ Mobile App...,12
33,Use the sections below to list the assets you ...,12
25,Please send liquidation requests directly to t...,12
59,For other types of non transferable assets you...,12


In [ ]:
print('There are', len(df_sent_topic.loc[df_sent_topic['dominant_topic'] == 4]), 'sentences that belong to topic 4 and we will remove')
print('There are', len(df_sent_topic.loc[df_sent_topic['dominant_topic'] == 12]), 'sentences that belong to topic 12 and we will remove')

There are 13 sentences that belong to topic 4 and we will remove
There are 9 sentences that belong to topic 12 and we will remove


In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Sentences")
df_topic_distribution.columns = ['Topic #', 'Num Sentences']
df_topic_distribution

,Topic #,Num Sentences
0,2,31
1,28,28
2,3,26
3,32,23
4,0,22
5,38,22
6,24,21
7,36,20
8,31,15
9,4,13


In [ ]:
df_sent_topic_clean = df_sent_topic.drop(df_sent_topic[(df_sent_topic.dominant_topic == 4) | (df_sent_topic.dominant_topic == 12)].index)

In [ ]:
df_description = df_sent_topic_clean[['sentence','CATEGORY']]
df_description = df_description.groupby('CATEGORY')['sentence'].agg(lambda col: ' '.join(col)).reset_index()

In [ ]:
df_description.head()

,CATEGORY,sentence
0,Ameritrade Institutional,A Transfer from an ACAT eligible Brokerage Fir...
1,HDFC,You can open a savings account with HDFC Bank ...
2,ICICI_LOMBARD,DuringÂ FY2019 we settledÂ 99 87 Â health insu...
3,Merrilllynch_CRA,For best results please complete the informati...
4,SBI,This product is a variant of Term Deposits but...


In [ ]:
df_description['sentence'][0]

'A Transfer from an ACAT eligible Brokerage Firm Bank Insurance Annuity Co Trust Co or Transfer Agent â Check box for full or partial transfer For partial transfers list descriptions of assets and shares Unless otherwise indicated TD Ameritrade will transfer in full Please note â The ACAT system does not allow for liquidation requests To place trades please contact your current custodian prior to submitting the transfer request Most banks insurance annuity and trust companies require original wet ink signature mailed to TD Ameritrade When transferring from a joint account at a contra firm to an individual account at TD Ameritrade the party losing ownership of the assets must sign section 4 If the discrepancy is a result of a name change for an account owner please provide a copy of the legal document such as a marriage certificate divorce decree etc Further documentation may be required depending on the situation If this account is a qualified retirement account I have amended the appl

In [ ]:
df_description.set_index('CATEGORY', inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_description['sentence'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df_description.index)
def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_category = []
  
    idx = indices[indices == name].index[0]

    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

  
    top_10_indexes = list(score_series.iloc[1:6].index)
    

    for i in top_10_indexes:
        recommended_category.append(list(df_description.index)[i])
        
    return recommended_category

In [ ]:
recommendations('Merrilllynch_CRA')

['Ameritrade Institutional', 'Schwab_IRA', 'SBI', 'HDFC ', 'ICICI_LOMBARD']

In [ ]:
recommendations('SBI')

['Schwab_IRA',
 'Merrilllynch_CRA',
 'Ameritrade Institutional',
 'ICICI_LOMBARD',
 'HDFC ']

In [ ]:
recommendations('ICICI_LOMBARD')

['SBI', 'Ameritrade Institutional', 'Schwab_IRA', 'HDFC ', 'Merrilllynch_CRA']